<a href="https://colab.research.google.com/github/InDataSight/GrowthPan/blob/main/ETLconceptworkflow1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.0 Setup

### 1.1 Folder structure

In [1]:
!mkdir ScrapyProfesiaRawData ScrapyProfesiaProcessedData ScrapyProfesiaLogs

### 1.2 Install libraries and modules

In [2]:
# Define requirements
requirements = """bs4
azure.storage.blob
"""

# Specify the file name
file_name = "requirements.txt"

# Write the text to the file
with open(file_name, "w") as file:
    file.write(requirements)

print(f"{file_name} created successfully with the following content:\n{requirements}")


requirements.txt created successfully with the following content:
bs4
azure.storage.blob



In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 11.0 MB/s eta 0:00:00


In [4]:
#!pip install azure.storage.blob
#!pip install azure.identity

In [5]:
import subprocess
import requests
from bs4 import BeautifulSoup
import json
import re
import datetime

#Run pip for defined modules in the requirements.txt
pip_install_result = subprocess.run(['pip', 'install', '-r', 'requirements.txt'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

#output results for the pip_install_result
if pip_install_result.returncode == 0:
    print("Pip install successful")
else:
    print("Pip install failed")


Pip install successful


### 1.3 Test

In [6]:
# Define unit testing
requirements = """def test_library_installation():
    try:
        import requests, google.colab
    except ImportError:
        assert False, "Required library is not installed"
    assert True
"""

# Specify the file name
file_name = "ScrapyProfesiaSetupTest.py"

# Write the text to the file
with open(file_name, "w") as file:
    file.write(requirements)

print(f"{file_name} created successfully with the following content:\n{requirements}")

ScrapyProfesiaSetupTest.py created successfully with the following content:
def test_library_installation():
    try:
        import requests, google.colab
    except ImportError:
        assert False, "Required library is not installed"
    assert True



In [7]:
!pytest ScrapyProfesiaSetupTest.py

======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0
rootdir: /content
plugins: langsmith-0.3.2, typeguard-4.4.1, anyio-3.7.1
collected 1 item                                                                                   

ScrapyProfesiaSetupTest.py .                                                                 [100%]

======================================== 1 passed in 1.29s =========================================


## 2.0 Extract Raw Data

In [8]:
LINK = 'https://www.profesia.sk/O4988508'
RAWFILE = '/content/ScrapyProfesiaRawData/O4988508.txt'
PROCESSEDFILE = '/content/ScrapyProfesiaProcessedData/O4988508P.json'

### 2.1 Single page - proof of concept

In [9]:
def download_and_save(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        #Convert the parsed html content to a string
        #html_string = str(soup)
        text_content = soup.get_text(separator='\n', strip=True) # Get the text content with newlines as separators

        # Create a dictionary to store the data
        #data = {"html_content": html_string}

        with open(filename, 'w', encoding='utf-8') as f:
            f.write(text_content)
        print(f"Successfully downloaded and saved to {filename}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading URL: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [10]:
download_and_save(LINK, RAWFILE)

Successfully downloaded and saved to /content/ScrapyProfesiaRawData/O4988508.txt


## 3.0 Transform Raw Data

In [11]:
def extract_data(inputfile,outputfile):

  with open(inputfile, 'r', encoding='utf-8') as f:
          text_content = f.read()
  text_content = re.sub(r'Hľadanie práce.*?Hľadanie práce', '', text_content, flags=re.DOTALL)

  if 'Odporučiť ponuku známemu' in text_content:
    text_content = text_content.split('Odporučiť ponuku známemu', 1)[0]
  else:
    text_content = text_content.split('Reagovať na ponuku', 1)[0]
  #initiate data structure
  data = {}

  #regex ID - 7 digit number - test if 7 digits
  data['ID'] = re.search(r'ID:\s*(\d+)', text_content).group(1)

  # data when job offer was published - month.day.year or day.month.year
  # test ...
  PublishedDateRaw = re.search(r'Dátum zverejnenia:\s*([\d\.]+)', text_content).group(1)
  PublishedDateObject = datetime.datetime.strptime(PublishedDateRaw, "%d.%m.%Y")
  data['PublishedDate'] = PublishedDateObject.strftime("%Y-%m-%dT00:00:00+00:00")

  #
  data['Location'] = re.search(r'lokalita:\s*(.+)', text_content).group(1)

  # Find text between 'Pozícia' and 'Spoločnosť'
  positions_text = re.search(r'Pozícia:\s*(.+?)(?=\nSpoločnosť:)', text_content, re.DOTALL).group(1)

  # Get valid lines, ignore ',', create
  positions = [line.strip() for line in positions_text.splitlines() if line.strip() and line.strip() != ',']
  data['Positions'] = positions
  data['Company'] = re.search(r'Spoločnosť:\s*(.+)', text_content).group(1)
  data['SalaryBrutto'] = re.search(r'Základná zložka mzdy \(brutto\):\s*(.+)', text_content).group(1)
  data['JobOfferText'] = text_content

  json_data = json.dumps(data, indent=4)

  with open(outputfile, 'w', encoding='utf-8') as f:
            f.write(json_data)
  print(f'Successfully downloaded and saved to {outputfile}')

In [12]:
extract_data(RAWFILE,PROCESSEDFILE)

Successfully downloaded and saved to /content/ScrapyProfesiaProcessedData/O4988508P.json


In [18]:
import requests

# Get the public IP address
public_ip = requests.get("https://api64.ipify.org").text
print("Google Colab Public IP:", public_ip)
#Add IP to FW@AzureBlobStorage
#Google Colab Public IP: 34.86.192.81
#Google Colab Public IP: 34.136.176.126
#Not a static IP|

Google Colab Public IP: 34.66.47.3


In [14]:
from google.colab import userdata
connection_string = userdata.get('connectionstring')

from azure.storage.blob import BlobServiceClient

container_name="profesiafulldata"
blob_name = "offer_4988508.txt"

In [15]:
def process_all_offers(connection_string, container_name):
    """
    Reads all 'offer_*.txt' files from Azure Blob Storage, processes them using
    extract_data_azure, and saves the output as JSON files.

    Args:
        connection_string (str): The connection string for your Azure Blob Storage account.
        container_name (str): The name of the container containing the offer files.
    """

    # Connect to the blob service
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    # List all blobs with the 'offer_' prefix
    blob_list = container_client.list_blobs(prefix="offer_")

    for blob in blob_list:
        if blob.name.endswith(".txt"):  # Process only .txt files
            # Download the blob content
            blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
            blob_content = blob_client.download_blob().readall().decode("utf-8")
            metadata = blob_client.get_blob_properties()

            # Construct output file name
            output_file_name = blob.name.replace(".txt", "P.json")
            output_file_path = f"/content/ScrapyProfesiaProcessedData/{output_file_name}"
            print(output_file_path)

            # Process the data and save to JSON
            #extract_data_azure(blob_content, metadata, output_file_path)

In [21]:
# Connect to the blob service
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# text
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download the blob content
#blob_data = blob_client.download_blob()

# Read data
blob_content = blob_client.download_blob().readall().decode("utf-8")

In [ ]:
# Get blob properties
# blob_properties = blob_client.get_blob_properties()

In [ ]:
# blob_properties

In [ ]:
#ExtractDate = blob_client.get_blob_properties()
#ExtractDate = blob_properties.get("creation_time")

In [ ]:
#timestamp_str = ExtractDate.isoformat()

In [ ]:
#ExtractDate

In [ ]:
#timestamp_str

In [ ]:
#ExtractDate1 = blob_client.get_blob_properties().get("creation_time").isoformat()
#ExtractDate1

In [22]:
RAWFILEAZURE = blob_content
METADATA = blob_client.get_blob_properties()
PROCESSEDFILEAZURE = '/content/ScrapyProfesiaProcessedData/offer_4988508P.json'

In [23]:
def extract_data_azure(inputfile,metadatafile,outputfile):

  text_content = re.sub(r'Hľadanie práce.*?Hľadanie práce', '', inputfile, flags=re.DOTALL)

  if 'Odporučiť ponuku známemu' in text_content:
    text_content = text_content.split('Odporučiť ponuku známemu', 1)[0]
  else:
    text_content = text_content.split('Reagovať na ponuku', 1)[0]

  data = {}
  data['ID'] = re.search(r'ID:\s*(\d+)', text_content).group(1)
  data['PublishedDate'] = re.search(r'Dátum zverejnenia:\s*([\d\.]+)', text_content).group(1)
  data['ExtractDate'] = metadatafile.get("creation_time").isoformat()
  data['Location'] = re.search(r'lokalita:\s*(.+)', text_content).group(1)
  # Find text between 'Pozícia' and 'Spoločnosť'
  positions_text = re.search(r'Pozícia:\s*(.+?)(?=\nSpoločnosť:)', text_content, re.DOTALL).group(1)
  # Get valid lines, ignore ',', create
  positions = [line.strip() for line in positions_text.splitlines() if line.strip() and line.strip() != ',']
  data['Positions'] = positions
  data['Company'] = re.search(r'Spoločnosť:\s*(.+)', text_content).group(1)
  data['SalaryBrutto'] = re.search(r'Základná zložka mzdy \(brutto\):\s*(.+)', text_content).group(1)
  data['JobOfferText'] = text_content

  json_data = json.dumps(data, indent=4)

  with open(outputfile, 'w', encoding='utf-8') as f:
            f.write(json_data)
  print(f'Successfully downloaded and saved to {outputfile}')

In [24]:
extract_data_azure(blob_content,METADATA,PROCESSEDFILEAZURE)

Successfully downloaded and saved to /content/ScrapyProfesiaProcessedData/offer_4988508P.json


In [25]:
!cat /content/ScrapyProfesiaProcessedData/offer_4988508P.json

{
    "ID": "4988508",
    "PublishedDate": "8.1.2025",
    "ExtractDate": "2025-01-09T10:03:08+00:00",
    "Location": "Bratislava",
    "Positions": [
        "Business Intelligence Specialist",
        "Database Analyst",
        "Programmer",
        "Python Programmer"
    ],
    "Company": "Swiss Re",
    "SalaryBrutto": "2 400 EUR/month",
    "JobOfferText": "Data Engineer - Swiss Re | PROFESIA.SK\n\nData Engineer\nAbout us\nOur benefits\nOpen positions\nData Engineer\nPlace of work\nTwin City B, Mlynsk\u00e9 nivy, Bratislava, Slovakia (Job with occasional home office)\nContract type\nfull-time\nWage (gross)\n2 400 - 4 100 EUR/month\nFor Slovakia the base salary range for this position is between [EUR 2,400] and [EUR 4,100] per month (for a full-time role). The specific salary offered considers: \u2022 the requirements, scope, complexity and responsibilities of the role, \u2022 the applicant\u2019s own profile including education/qualifications, expertise, specialization, skills

In [26]:
!cat /content/ScrapyProfesiaProcessedData/O4988508P.json

{
    "ID": "4988508",
    "PublishedDate": "2025-01-08T00:00:00+00:00",
    "Location": "Bratislava",
    "Positions": [
        "Business Intelligence Specialist",
        "Database Analyst",
        "Programmer",
        "Python Programmer"
    ],
    "Company": "Swiss Re",
    "SalaryBrutto": "2 400 EUR/month",
    "JobOfferText": "Data Engineer - Swiss Re | PROFESIA.SK\n\nData Engineer\nAbout us\nOur benefits\nOpen positions\nData Engineer\nPlace of work\nTwin City B, Mlynsk\u00e9 nivy, Bratislava, Slovakia (Job with occasional home office)\nContract type\nfull-time\nWage (gross)\n2 400 - 4 100 EUR/month\nFor Slovakia the base salary range for this position is between [EUR 2,400] and [EUR 4,100] per month (for a full-time role). The specific salary offered considers: \u2022 the requirements, scope, complexity and responsibilities of the role, \u2022 the applicant\u2019s own profile including education/qualifications, expertise, specialization, skills and experience. In addition to

In [27]:
!ls -aGl /content/ScrapyProfesiaProcessedData/

total 24
drwxr-xr-x 2 root 4096 Feb  3 12:37 .
drwxr-xr-x 1 root 4096 Feb  3 12:30 ..
-rw-r--r-- 1 root 4803 Feb  3 12:33 O4988508P.json
-rw-r--r-- 1 root 4834 Feb  3 12:37 offer_4988508P.json


In [ ]:
#!ping 95.105.175.243

In [ ]:
#import requests

#try:
#    response = requests.get('http://95.105.175.243')  # Replace with your IP address
#    if response.status_code == 200:
#        print("The IP is reachable.")
#    else:
#        print(f"Server responded with status code: {response.status_code}")
#except requests.exceptions.RequestException as e:
#    print(f"Error connecting to the IP: {e}")
